In [10]:
# Training Parameters
learning_rate = 0.0001
epochs = 20
logs_path = 'tensorboard_logs'

# Network Parameters
n_classes = np.unique(y_train).shape[0] # 43 traffic sign classes/labels there are in the dataset.
dropout = 0.8  # Dropout, probability to keep units

In [11]:
def conv2fc(input_height, input_width, filter_size, stride, layers):
    
    con2fc_height = input_height
    con2fc_width = input_width
    for i in layers:
        con2fc_height = np.ceil((con2fc_height - filter_size[i] + 1) / stride[i])
        con2fc_width = np.ceil((con2fc_width - filter_size[i] + 1) / stride[i])
    return int(con2fc_height) * int(con2fc_width)

In [12]:
layers = ['conv_1', 'maxpool_1', 'conv_2', 'maxpool_2']

# (img shape: 32*32)
layer_input = {
     'depth': 1
    ,'height': X_train.shape[1]
    ,'width': X_train.shape[2]
}

layer_depth = {
    'conv_1': 32
    ,'conv_2': 128
    ,'fc_1': 1024
    ,'fc_2': 1024
    ,'out': n_classes
}

# a kernel size of 3
filter_size={
    'conv_1': 5
    ,'maxpool_1': 2
    ,'conv_2': 5
    ,'maxpool_2': 2
}

stride = {
    'conv_1': 1
    ,'maxpool_1': 2
    ,'conv_2': 1
    ,'maxpool_2': 2
}

# xavier_initializer
# ref https://www.tensorflow.org/api_docs/python/tf/contrib/layers/xavier_initializer
# tf.truncated_normal(shape=[dims[l-1],dims[l]], mean=mu[l], stddev=std[l], dtype=tf.float64)
    
for k,v in sorted(layer_depth.items()):
    print("Layer %5s depth is %s"%(k,v))

Layer conv_1 depth is 32
Layer conv_2 depth is 128
Layer  fc_1 depth is 1024
Layer  fc_2 depth is 1024
Layer   out depth is 43


### Architecture

In [13]:
# Tensorboard graph
def conv2d(x, W, b, stride):
    x = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='VALID')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

def maxpool2d(x, k, s):
    # Tensor input is 4-D: [Batch Size, Height, Width, Feature(Channel)]
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding='VALID')

def fullycon(x, w, b):
    return tf.nn.relu(tf.nn.xw_plus_b(tf.contrib.layers.flatten(x), w, b))

def conv_net(x, weights, biases, dropout):
    # Input
    print ("input shape is: " + str(x.shape))
    
    # Layer 1 
    conv1 = conv2d(x, weights['conv1'], biases['conv1'], stride['conv_1'])
    print ("conv1 shape is: " + str(conv1.shape))

    conv1 = maxpool2d(conv1, filter_size['maxpool_1'], stride['maxpool_1'])
    print ("maxpool1 shape is: " + str(conv1.shape))

    # Layer 2 
    conv2 = conv2d(conv1, weights['conv2'], biases['conv2'], stride['conv_2'])
    print ("conv2 shape is: " + str(conv2.shape))
    
    conv2 = maxpool2d(conv2, filter_size['maxpool_1'], stride['maxpool_2'])
    print ("maxpool2 shape is: " + str(conv2.shape))
    
    # Layer 3
    fc1 = fullycon(conv2, weights['fc1'], biases['fc1'])
    print ("fully connected layer from conv2 {} to fc1 {}".format(conv2.shape,fc1.shape))
    
    fc1 = tf.nn.dropout(fc1, dropout)
    print ("fully connected layer 1 dropout is: {}".format(fc1.shape))
    
    # Layer 4
    fc2 = fullycon(fc1, weights['fc2'], biases['fc2'])
    print ("fully connected layer from fc1 {} to fc2 {}".format(fc2.shape,fc1.shape))
    
    fc2 = tf.nn.dropout(fc2, dropout)
    print ("fully connected layer 2 dropout is: {}".format(fc2.shape))

    # Layer 5: output
    out = tf.nn.xw_plus_b(fc2, weights['out'], biases['out'])
    print ("output shape is: {}".format(out.shape))

    return out

In [14]:
"""
graph computation model
"""

# ref: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/4_Utils/tensorboard_basic.py
# tf Graph input

tf.reset_default_graph() 
# g = tf.Graph()
# with g.as_default():
x = tf.placeholder(tf.float32, [None, layer_input["height"], layer_input["width"], 1], name='InputData')
y = tf.placeholder(tf.float32, [None, n_classes], name='LabelData')
keep_prob = tf.placeholder(tf.float32)

conv2fc_num = conv2fc(layer_input["height"], layer_input["width"], filter_size, stride, layers)
weights = {
    'conv1': tf.get_variable("conv1w", shape=[filter_size['conv_1'], filter_size['conv_1']
                                              , layer_input['depth'], layer_depth['conv_1']]
                             , initializer=tf.contrib.layers.xavier_initializer()),
    'conv2': tf.get_variable("conv2w", shape=[filter_size['conv_2'], filter_size['conv_2']
                                              , layer_depth['conv_1'], layer_depth['conv_2']]
                             , initializer=tf.contrib.layers.xavier_initializer()),
    'fc1':  tf.get_variable("fc1w", shape=[conv2fc_num * layer_depth['conv_2'], layer_depth['fc_1']]
                            , initializer=tf.contrib.layers.xavier_initializer()),
    'fc2':  tf.get_variable("fc2w", shape=[layer_depth['fc_1'], layer_depth['fc_2']]
                            , initializer=tf.contrib.layers.xavier_initializer()),
    'out':  tf.get_variable("out1w", shape=[layer_depth['fc_2'], layer_depth['out']]
                            , initializer=tf.contrib.layers.xavier_initializer())
}

# ref https://www.leiphone.com/news/201703/3qMp45aQtbxTdzmK.html
biases = {
    'conv1': tf.get_variable("conv1b", shape=[layer_depth['conv_1']], initializer=tf.contrib.layers.xavier_initializer()),
    'conv2': tf.get_variable("conv2b", shape=[layer_depth['conv_2']], initializer=tf.contrib.layers.xavier_initializer()),
    'fc1': tf.get_variable("fc1b", shape=[layer_depth['fc_1']], initializer=tf.contrib.layers.xavier_initializer()),
    'fc2': tf.get_variable("fc2b", shape=[layer_depth['fc_2']], initializer=tf.contrib.layers.xavier_initializer()),
    'out': tf.get_variable("out1b", shape=[layer_depth['out']], initializer=tf.contrib.layers.xavier_initializer())
}

for k,v in sorted(weights.items()):
    print("Weights for %5s: %s"%(k,v.shape))

# with tf.name_scope('Model'):
logits = conv_net(x, weights, biases, keep_prob)

# with tf.name_scope('Cost'):
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


# with tf.name_scope('Accuracy'):
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", accuracy)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

Weights for conv1: (5, 5, 1, 32)
Weights for conv2: (5, 5, 32, 128)
Weights for   fc1: (3200, 1024)
Weights for   fc2: (1024, 1024)
Weights for   out: (1024, 43)
input shape is: (?, 32, 32, 1)
conv1 shape is: (?, 28, 28, 32)
maxpool1 shape is: (?, 14, 14, 32)
conv2 shape is: (?, 10, 10, 128)
maxpool2 shape is: (?, 5, 5, 128)
fully connected layer from conv2 (?, 5, 5, 128) to fc1 (?, 1024)
fully connected layer 1 dropout is: (?, 1024)
fully connected layer from fc1 (?, 1024) to fc2 (?, 1024)
fully connected layer 2 dropout is: (?, 1024)
output shape is: (?, 43)


### Data preprocessing

In [15]:
def BatchClaheEqualizer(imgs, method, tileSize = 8, clipLim = 2.0):
    sample_num = imgs.shape[0]
    final_imgs = np.zeros((imgs.shape[0],imgs.shape[1],imgs.shape[2]), dtype=np.int)
    
    if method == "clahe":    
        clahe = cv2.createCLAHE(clipLimit=clipLim, tileGridSize=(tileSize,tileSize))
        for i in range(sample_num):
            final_imgs[i] = clahe.apply(cv2.cvtColor(imgs[i], cv2.COLOR_RGB2GRAY))
        return np.expand_dims(final_imgs, axis=-1)
    elif method == "hist":
        for i in range(sample_num):
            final_imgs[i] = cv2.equalizeHist(cv2.cvtColor(imgs[i], cv2.COLOR_RGB2GRAY))
        return np.expand_dims(final_imgs, axis=-1)
    else:    
        for i in range(sample_num):
            final_imgs[i] = cv2.cvtColor(imgs[i], cv2.COLOR_RGB2GRAY)
        return imgs

In [17]:
X_train_equ = BatchClaheEqualizer(X_train, "clahe")
X_train_input = image_Normalization(X_train_equ)

X_valid_equ = BatchClaheEqualizer(X_valid, "clahe")
X_valid_input = image_Normalization(X_valid_equ)

X_test_equ = BatchClaheEqualizer(X_test, "clahe")
X_test_input = image_Normalization(X_test_equ)

In [19]:
def OneHotEncoding(y_class):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_class)
    return lb

In [20]:
ohe = OneHotEncoding(signNames_pd["ClassId"])

In [21]:
y_train_input = ohe.transform(y_train)
y_valid_input = ohe.transform(y_valid)
y_test_input = ohe.transform(y_test)

### train, evaluation and test

In [22]:
import time

In [24]:
# Initializing the variables
init = tf.global_variables_initializer()

# tf.summary.FileWriter("logs", g).close()

In [25]:
start_time = time.time()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(epochs):
        
        # get batch generator
        bgss = batch_Generator_StratifiedSampling(y_train_dist, X_train_input, y_train_input)
        get_nextBatch = bgss.batches()
        
        batch_id = 0
        for _, train_index in get_nextBatch:  # for batch_id, batch_x, batch_y in batch_iteration(X_train_input, y_train_input, batch_size):
            batch_id += 1
            batch_x = X_train_input[train_index]
            batch_y = y_train_input[train_index]
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            
        valid_acc = sess.run(accuracy, feed_dict={
            x: X_valid_input,
            y: y_valid_input,
            keep_prob: 1.})
        print('Epoch {:>2}/{}, Batch {:>3}/{} - Loss: {:>10.1f} Validation Accuracy: {:.6f}'.format(
            epoch + 1,
            epochs,
            batch_id,
            bgss.num_split,
            loss,
            valid_acc))

        # Calculate Test Accuracy
        test_acc = sess.run(accuracy, feed_dict={
            x: X_test_input,
            y: y_test_input,
            keep_prob: 1.})
        print('Testing Accuracy: {}'.format(test_acc))
    
print("--- %s seconds ---" % (time.time() - start_time))

Epoch  1/20, Batch 610/610 - Loss:        3.8 Validation Accuracy: 0.036508
Testing Accuracy: 0.03958828002214432
Epoch  2/20, Batch 610/610 - Loss:        3.8 Validation Accuracy: 0.047619
Testing Accuracy: 0.05225653201341629
Epoch  3/20, Batch 610/610 - Loss:        3.7 Validation Accuracy: 0.047619
Testing Accuracy: 0.05225653201341629
Epoch  4/20, Batch 610/610 - Loss:        3.7 Validation Accuracy: 0.047619
Testing Accuracy: 0.05225653201341629
Epoch  5/20, Batch 610/610 - Loss:        3.7 Validation Accuracy: 0.047619
Testing Accuracy: 0.05225653201341629
Epoch  6/20, Batch 610/610 - Loss:        3.7 Validation Accuracy: 0.047619
Testing Accuracy: 0.05225653201341629
Epoch  7/20, Batch 610/610 - Loss:        3.7 Validation Accuracy: 0.047619
Testing Accuracy: 0.05225653201341629
Epoch  8/20, Batch 610/610 - Loss:        3.7 Validation Accuracy: 0.047619
Testing Accuracy: 0.05225653201341629
Epoch  9/20, Batch 610/610 - Loss:        3.7 Validation Accuracy: 0.047846
Testing Accu

---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

### Predict the Sign Type for Each Image

In [3]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

### Analyze Performance

In [4]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 

The example below demonstrates how tf.nn.top_k can be used to find the top k predictions for each image.

`tf.nn.top_k` will return the values and indices (class ids) of the top k predictions. So if k=3, for each sign, it'll return the 3 largest probabilities (out of a possible 43) and the correspoding class ids.

Take this numpy array as an example. The values in the array represent predictions. The array contains softmax probabilities for five candidate images with six possible classes. `tf.nn.top_k` is used to choose the three classes with the highest probability:

```
# (5, 6) array
a = np.array([[ 0.24879643,  0.07032244,  0.12641572,  0.34763842,  0.07893497,
         0.12789202],
       [ 0.28086119,  0.27569815,  0.08594638,  0.0178669 ,  0.18063401,
         0.15899337],
       [ 0.26076848,  0.23664738,  0.08020603,  0.07001922,  0.1134371 ,
         0.23892179],
       [ 0.11943333,  0.29198961,  0.02605103,  0.26234032,  0.1351348 ,
         0.16505091],
       [ 0.09561176,  0.34396535,  0.0643941 ,  0.16240774,  0.24206137,
         0.09155967]])
```

Running it through `sess.run(tf.nn.top_k(tf.constant(a), k=3))` produces:

```
TopKV2(values=array([[ 0.34763842,  0.24879643,  0.12789202],
       [ 0.28086119,  0.27569815,  0.18063401],
       [ 0.26076848,  0.23892179,  0.23664738],
       [ 0.29198961,  0.26234032,  0.16505091],
       [ 0.34396535,  0.24206137,  0.16240774]]), indices=array([[3, 0, 5],
       [0, 1, 4],
       [0, 5, 1],
       [1, 3, 5],
       [1, 4, 3]], dtype=int32))
```

Looking just at the first row we get `[ 0.34763842,  0.24879643,  0.12789202]`, you can confirm these are the 3 largest probabilities in `a`. You'll also notice `[3, 0, 5]` are the corresponding indices.

In [3]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin =activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")